In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from z3 import *
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from PIL import Image

mnist = fetch_openml('mnist_784', version=1, cache=True, parser='auto')
X_iris, y_iris = load_iris(return_X_y=True, as_frame=True)

scaler = MinMaxScaler()
scaler.fit(mnist.data)
scaler.set_output(transform='pandas')
data_scaled = scaler.transform(mnist.data)

scaler.fit(X_iris)
scaler.set_output(transform='pandas')
X_iris = scaler.transform(X_iris)